In [1]:
!pip install hazm
!pip install stopwords_guilannlp
!pip install -U -q PyDrive

     |████████████████████████████████| 316 kB 8.5 MB/s 
     |████████████████████████████████| 233 kB 61.8 MB/s 
     |████████████████████████████████| 1.4 MB 52.8 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=54c22983d77d20b7d872a15986e536d895f84831309b808c328dcb72408e2c08
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154231 sha256=2c2607f442d47b385e031790956b5096edbe47f2a47a5e1c2970be3c1fe3ec2f
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [26]:

import json
import nltk
import numpy as np
from hazm import *
import pandas as pd
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import collections
import re
from collections import Counter, defaultdict
from stopwords_guilannlp import stopwords_output
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

In [27]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
link = 'https://drive.google.com/file/d/1TxrM59TyQjJfB3oh-qK4TOYb1aQ6Ds4C/view?usp=sharing' # The shareable link
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('asriran.jsonl')
# Dataset is now stored in a Pandas Dataframe

sharing


In [28]:

file = open('asriran.jsonl', 'r')
asriran = file.readlines()


print('There is {} item available'.format(len(asriran)))

There is 130 item available


In [29]:
def asriran_prepare_json(j):
  # clear newsPath
  newsPath = j['newsPath']
  newsPath = newsPath.replace('عصرايران دو » ','')
  newsPath = newsPath.replace('صفحه نخست » ','')
  j['newsPath'] = newsPath
  return j

raw_data = []
agency_data = []
for item in asriran:
  try:
    j = json.loads(item)
    asriran_prepare_json(j)
    raw_data.append({'title': j['title'], 'body':j['body'], 'newsPath':j['newsPath'], 'tag':list(j['tags'].keys())[0] })
    agency_data.append({'title': j['title'], 'body':j['body'], 'newsAgency': j['NewsAgency']})
  except:
    pass


In [33]:
category_df = pd.DataFrame(raw_data)
category_df.head(200)

,title,body,newsPath,tag
0,وزیر بهداشت خواستار جلوگیری از تاسیس کارخانه ا...,وزیر بهداشت، درمان و آموزش پزشکی در نامه&zwnj;...,اجتماعی,وزیر بهداشت
1,همایون امیرزاده رئیس مرکز روابط عمومی وزارت ار...,​عباس صالحی وزیر فرهنگ و ارشاد اسلامی در حکمی ...,فرهنگی/هنری,ارشاد
2,فتحی: تلاش می‌کنم استقلال را به جایگاه اصلی خو...,سرپرست جدید باشگاه استقلال می&zwnj;گوید تلاش م...,ورزشی,استقلال
3,بانک اطلاعات مجرمان حرفه ای راه اندازی می شود,رئیس قوه قضائیه دستورالعملی را برای اجرا ابلاغ...,اجتماعی,مجرمان حرفه ای
4,ارسال ۷۰۰ پرونده درباره واردات خودرو به سازمان...,رییس گشت&zwnj;های مشترک تعزیرات استان تهران گف...,اقتصادی,تعزیرات
...,...,...,...,...
125,تلاش فدراسیون فوتبال الجزایر برای جذب کی‌روش,فدراسیون فوتبال الجزایر برای به خدمت گرفتن سرم...,ورزشی,کی روش
126,اس ام اس شب جمعه و فاتحه برای اموات (3),ما هم گذشتگان فردا هستيم امروز كه هستيم با فات...,خواندنی ها و دیدنی ها,اس ام اس شب جمعه
127,"بهترین های دنیای خودرو در فستیوال "" گوودوود"" (...",تابستان همواره بهانه خوبی برای تمام مردم جهان ...,اقتصادی,فستیوال
128,دیدنی های امروز,"بازدید رهبر کره شمالی از کارخانه کشتی‌سازی ""چو...",عمومی,دیدنی های امروز


In [9]:
agency_df = pd.DataFrame(agency_data)
agency_df.head()

,title,body,newsAgency
0,وزیر بهداشت خواستار جلوگیری از تاسیس کارخانه ا...,وزیر بهداشت، درمان و آموزش پزشکی در نامه&zwnj;...,AsrIran
1,همایون امیرزاده رئیس مرکز روابط عمومی وزارت ار...,​عباس صالحی وزیر فرهنگ و ارشاد اسلامی در حکمی ...,AsrIran
2,فتحی: تلاش می‌کنم استقلال را به جایگاه اصلی خو...,سرپرست جدید باشگاه استقلال می&zwnj;گوید تلاش م...,AsrIran
3,بانک اطلاعات مجرمان حرفه ای راه اندازی می شود,رئیس قوه قضائیه دستورالعملی را برای اجرا ابلاغ...,AsrIran
4,ارسال ۷۰۰ پرونده درباره واردات خودرو به سازمان...,رییس گشت&zwnj;های مشترک تعزیرات استان تهران گف...,AsrIran


In [10]:
np.unique(category_df['newsPath'])

array(['اجتماعی', 'اقتصادی', 'بین الملل', 'حوادث',
       'خواندنی ها و دیدنی ها', 'داستان کوتاه', 'روانشناسی', 'سلامت',
       'سیاست خارجی', 'سیاسی', 'علمی', 'عمومی', 'فرهنگی/هنری',
       'فناوری و IT', 'ورزشی'], dtype=object)

In [11]:
category_le = LabelEncoder()
category_df['newsPath'] = category_le.fit_transform(category_df['newsPath'])

agency_le = LabelEncoder()
agency_df['newsAgency'] = agency_le.fit_transform(agency_df['newsAgency'])

category_x = category_df[['title','body','tag']]

category_y = category_df['newsPath']

agency_x = agency_df[['title','body']]

agency_y = agency_df['newsAgency']

In [12]:
category_x.head()

,title,body,tag
0,وزیر بهداشت خواستار جلوگیری از تاسیس کارخانه ا...,وزیر بهداشت، درمان و آموزش پزشکی در نامه&zwnj;...,وزیر بهداشت
1,همایون امیرزاده رئیس مرکز روابط عمومی وزارت ار...,​عباس صالحی وزیر فرهنگ و ارشاد اسلامی در حکمی ...,ارشاد
2,فتحی: تلاش می‌کنم استقلال را به جایگاه اصلی خو...,سرپرست جدید باشگاه استقلال می&zwnj;گوید تلاش م...,استقلال
3,بانک اطلاعات مجرمان حرفه ای راه اندازی می شود,رئیس قوه قضائیه دستورالعملی را برای اجرا ابلاغ...,مجرمان حرفه ای
4,ارسال ۷۰۰ پرونده درباره واردات خودرو به سازمان...,رییس گشت&zwnj;های مشترک تعزیرات استان تهران گف...,تعزیرات


In [13]:
category_y

0       0
1      12
2      14
3       0
4       1
       ..
125    14
126     4
127     1
128    11
129    14
Name: newsPath, Length: 130, dtype: int64